<a href="https://colab.research.google.com/github/Maxim687/Machine-learning/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0_4_1_%D0%A2%D0%B8%D0%BC%D0%BE%D1%89%D1%83%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Завдання 1
# Попередній аналіз даних.

file_path = '/content/drive/My Drive/titanic (2).csv'
df = pd.read_csv(file_path)

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
df.shape

(418, 12)

In [ ]:
df.dtypes

,0
PassengerId,int64
Survived,int64
Pclass,int64
Name,object
Sex,object
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64


In [ ]:
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns

df[numeric_columns] = df[numeric_columns].apply(lambda x: x.fillna(x.mean()))

In [ ]:
df.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,0
SibSp,0
Parch,0
Ticket,0
Fare,0


In [ ]:
duplicate_count = df.duplicated().sum()

df.drop_duplicates(inplace=True)

duplicate_count_after = df.duplicated().sum()

duplicate_count, duplicate_count_after

(0, 0)

In [ ]:
df_selected = df[['Survived', 'Pclass', 'Sex', 'Age', 'Fare']]

df_selected.head()

df_selected['Sex'].unique()

df_selected.loc[df_selected['Sex'] == 'male', 'Sex'] = 0
df_selected.loc[df_selected['Sex'] == 'female', 'Sex'] = 1

df_selected['Sex'].unique()

array([0, 1], dtype=object)

In [ ]:
# Завдання 2
# Будуємо 3 класифікатори на вибір, визначаємо оптимальні параметри (оптимізатор - на вибір).

from sklearn.model_selection import train_test_split

X = df_selected[['Pclass', 'Sex', 'Age', 'Fare']]
y = df_selected['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

log_reg = LogisticRegression()
tree_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier()

param_grid_lr = {'C': [0.01, 0.1, 1, 10], 'solver': ['liblinear']}
param_grid_tree = {'max_depth': [3, 5, 10, 20], 'min_samples_split': [2, 5, 10]}
param_grid_rf = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10]}

grid_search_lr = GridSearchCV(log_reg, param_grid_lr, cv=5, scoring='accuracy')
grid_search_tree = GridSearchCV(tree_clf, param_grid_tree, cv=5, scoring='accuracy')
grid_search_rf = GridSearchCV(rf_clf, param_grid_rf, cv=5, scoring='accuracy')

grid_search_lr.fit(X_train, y_train)
grid_search_tree.fit(X_train, y_train)
grid_search_rf.fit(X_train, y_train)

best_lr_model = grid_search_lr.best_estimator_
best_tree_model = grid_search_tree.best_estimator_
best_rf_model = grid_search_rf.best_estimator_

print("Найкращі параметри для Логістичної регресії:", grid_search_lr.best_params_)
print("Найкращі параметри для Дерева рішень:", grid_search_tree.best_params_)
print("Найкращі параметри для Випадкового лісу:", grid_search_rf.best_params_)

Найкращі параметри для Логістичної регресії: {'C': 0.1, 'solver': 'liblinear'}
Найкращі параметри для Дерева рішень: {'max_depth': 3, 'min_samples_split': 2}
Найкращі параметри для Випадкового лісу: {'max_depth': 3, 'n_estimators': 10}


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred_lr = best_lr_model.predict(X_test)
y_pred_tree = best_tree_model.predict(X_test)
y_pred_rf = best_rf_model.predict(X_test)

print("Точність логістичної регресії:", accuracy_score(y_test, y_pred_lr))
print("Точність дерева рішень:", accuracy_score(y_test, y_pred_tree))
print("Точність випадкового лісу:", accuracy_score(y_test, y_pred_rf))

print("\nЗвіт про класифікацію для логістичної регресії:")
print(classification_report(y_test, y_pred_lr))

print("\nЗвіт про класифікацію для дерева рішень:")
print(classification_report(y_test, y_pred_tree))

print("\nЗвіт про класифікацію для випадкового лісу:")
print(classification_report(y_test, y_pred_rf))

print("\nМатриця помилок випадкового лісу:")
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
print(conf_matrix_rf)

Точність логістичної регресії: 1.0
Точність дерева рішень: 1.0
Точність випадкового лісу: 1.0

Звіт про класифікацію для логістичної регресії:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        34

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84


Звіт про класифікацію для дерева рішень:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        34

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84


Звіт про класифікацію для випадкового лісу:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
       

In [ ]:
import numpy as np

random_indices = np.random.choice(len(X_test), 10, replace=False)
X_random = X_test.iloc[random_indices]
y_random_pred = best_rf_model.predict(X_random)

print("\nПрогноз для 10 випадкових прикладів:")
print(y_random_pred)


Прогноз для 10 випадкових прикладів:
[1 1 0 0 0 0 0 0 0 0]
